In [63]:
import pandas as pd
import pickle

from imblearn.over_sampling import RandomOverSampler
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

In [42]:
df_sum_NjNy_2019 = pd.read_csv('/Users/ridwan/Documents/DataScience/Metis/Metis_Projects/Data/flights/sum_NjNy_2019.csv')

In [45]:
df_sum_NjNy_2019.columns

Index(['DAY_OF_WEEK', 'FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'DEST', 'DEST_CITY_NAME', 'CRS_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
       'LATE_AIRCRAFT_DELAY', 'DELAY'],
      dtype='object')

In [46]:
#Binary classificaton
df_sum_NjNy_2019['DELAY'] = ['DELAY' if x >= 1 else 'ONTIME' for x in df_sum_NjNy_2019['DEP_DELAY']]

In [47]:
#Dropping delay columns; save for later potentially
df_sum_NjNy_2019_delay = df_sum_NjNy_2019.drop(df_sum_NjNy_2019.loc[:,'CARRIER_DELAY':'LATE_AIRCRAFT_DELAY'].columns, axis=1)
df_sum_NjNy_2019_delay = df_sum_NjNy_2019_delay.dropna()

df_sum_NjNy_2019_delay['DAY_OF_WEEK'] = df_sum_NjNy_2019_delay['DAY_OF_WEEK'].replace({1: 'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday', 6:'Saturday', 7:'Sunday'})

df_sum_NjNy_2019_delay['OP_CARRIER'] = df_sum_NjNy_2019_delay['OP_CARRIER'].replace({'B6':'Jetblue', 'UA':'United', 'AS':'Alaska', 'YV':'Mesa', 'EV':'AtlanticSouthest', 'OO':'Skywest', 'AA':'AmericanAirlines', 'MQ':'AmericanEagle', '9E':'Endeavor', 'G4':'Allegiant', 'F9':'Frontier', 'YX':'Republic', 'NK':'Spirit', 'OH':'PSA', 'HA':'Hawaiian', 'WN':'Southwest', 'DL':'Delta'})

# Changing Airline string names to numerical 
# df_sum_NjNy_2019_delay['OP_CARRIER'] = df_sum_NjNy_2019_delay['OP_CARRIER'].replace({'B6':1, 'UA':2, 'AS':3, 'YV':4, 'EV':5, 'OO':6, 'AA':7, 'MQ':8, '9E':9, 'G4':10, 'F9':11, 'YX':12, 'NK':13, 'OH':14, 'HA':15, 'WN':16, 'DL':17})

# Changing Origin string names to numerical 
# df_sum_NjNy_2019_delay['ORIGIN'] = df_sum_NjNy_2019_delay['ORIGIN'].replace({'JFK':1, 'LGA' : 2, 'EWR' : 3})

# Changing Destination string names to numerical 
# df_sum_NjNy_2019_delay['DEST'] = df_sum_NjNy_2019_delay['DEST'].replace({'LAX':1, 'DEN':2, 'SLC':3, 'SMF':4, 'DFW':5, 'AUS':6, 'MIA':7, 'MSY':8, 'CHS':9, 'BOS':10, 'MCO':11, 'RSW':12, 'RDU':13, 'SFO':14, 'ORD':15, 'LAS':16, 'IAH':17, 'MCI':18, 'TPA':19, 'SNA':20, 'ATL':21, 'PBI':22, 'SAN':23, 'FLL':24, 'MYR':25, 'PDX':26, 'SJU':27, 'STT':28, 'BNA':29, 'CLE':30, 'BQN':31, 'SRQ':32, 'IAD':33, 'PHX':34, 'HNL':35,'SAT':36, 'SJC':37, 'PVD':38, 'SAV':39, 'GSO':40, 'CLT':41, 'CMH':42, 'CVG':43, 'RIC':44,'IND':45, 'ORF':46, 'DTW':47, 'ROC':48, 'CHA':49, 'PWM':50, 'BTV':51, 'ABQ':52, 'BUF':53, 'SYR':54, 'LGB':55, 'ONT':56, 'BUR':57, 'HOU':58, 'DCA':59, 'ILM':60, 'TYS':61, 'PHL':62,  'PIT':63, 'DAY':64, 'BWI':65, 'CHO':66, 'MSP':67, 'GSP':68, 'OMA':69, 'MKE':70, 'AVL':71, 'BHM':72, 'SDF':73, 'STL':74, 'EYW':75, 'MSN':76, 'JAX':77, 'GRR':78, 'MEM':79, 'BGR':80, 'MDW':81, 'DAL':82, 'ROA':83, 'CAK':84, 'DSM':85, 'LEX':86, 'ORH':87, 'PSE':88, 'ACK':89,'OKC':90, 'HHH':91, 'CAE':92, 'DAB':93, 'VPS':94, 'RNO':95, 'OAK':96, 'MVY':97, 'XNA':98, 'MHT':99, 'TVC':100, 'BZN':101, 'PNS':102, 'JAC':103, 'HYA':104, 'ANC':105, 'RAP':106, 'SBN':107, 'SEA':108})

df_sum_NjNy_2019_delay['CRS_DEP_TIME'] = pd.to_datetime(df_sum_NjNy_2019_delay['CRS_DEP_TIME'], format='%H%M%S').dt.time

# df_sum_NjNy_2019_delay = df_sum_NjNy_2019_delay.groupby(df_sum_NjNy_2019_delay.CRS_DEP_TIME.dt.floor('30min')).mean()

#ACTUAL_ELAPSED_TIME drops KNN score significantly
X = df_sum_NjNy_2019_delay.drop(['DELAY','ORIGIN_CITY_NAME','DEST_CITY_NAME','DEP_DELAY','ARR_DELAY','OP_CARRIER_FL_NUM','FL_DATE','ARR_TIME','CRS_ARR_TIME','DEP_TIME','CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME','DISTANCE'], axis=1)

y = df_sum_NjNy_2019_delay['DELAY']

In [48]:
X

,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME
0,Monday,Alaska,JFK,SEA,07:00:00
1,Monday,Alaska,EWR,SEA,17:01:05
2,Monday,Alaska,EWR,SEA,06:00:00
3,Monday,Alaska,EWR,PDX,15:04:05
4,Monday,Alaska,JFK,LAX,13:00:00
...,...,...,...,...,...
111943,Sunday,Alaska,EWR,SEA,09:00:00
111944,Sunday,Alaska,JFK,LAX,11:03:00
111945,Sunday,Alaska,JFK,SEA,18:05:05
111946,Sunday,Alaska,JFK,LAX,07:03:00


In [49]:
list_day = list(X['DAY_OF_WEEK'].unique())
list_carrier = list(X['OP_CARRIER'].unique())
list_origin = list(X['ORIGIN'].unique())
list_destination = list(X['DEST'].unique())

In [50]:
f = open("./pickle/list_day", "wb")
pickle.dump(list_day, f)
f.close()

In [51]:
f = open("./pickle/list_carrier", "wb")
pickle.dump(list_carrier, f)
f.close()

In [52]:
f = open("./pickle/list_origin", "wb")
pickle.dump(list_origin, f)
f.close()

In [53]:
f = open("./pickle/list_destination", "wb")
pickle.dump(list_destination, f)
f.close()

In [60]:
X_ROS, y_ROS = RandomOverSampler().fit_sample(X,y)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_ROS,y_ROS, stratify=y_ROS)

In [64]:
# fit our model and score it on the test set
rfc_model = RandomForestClassifier()
rfc_model.fit(X_train, y_train)

ValueError: could not convert string to float: 'Thursday'